In [85]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import *
from pandas import DataFrame
import datetime


In [86]:
## Read in match dataset
df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')
df['year'] = pd.DatetimeIndex(df['match_date']).year

## Filter to only games from 2010 onwards. Remove shortened season stats (2019 preds and 2020 stats)

df = df.query('year >= 2012')\
        
## Read in Player Database
players = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\player_ages.csv')

## Merge game stats with player info
df['player_name'] = df['player_last_name'].astype(str) + ", " + df['player_first_name']
joined_data = df.merge(players, on=['player_name'],how='left')

## Calculate age on match day (take year of game less year of birth, so will remain same age all season)
joined_data['age'] = pd.DatetimeIndex(joined_data['match_date']).year - pd.DatetimeIndex(joined_data['dob']).year

# Write to csv to check
# joined_data.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl_limited.csv')

# Aggregate 
avg_scores = joined_data.groupby(['player_team', 'player_first_name', 'player_last_name','player_name', 'age', 'year'])\
.agg({'afl_fantasy_score': 'mean'})\
.sort_values(by=['afl_fantasy_score'], ascending=False)\

pd.set_option('display.max_rows', 10)
# print(avg_scores)
avg_scores.query('age < 22').head(10)

C:\Users\anoble\AppData\Local\Temp\ipykernel_41556\573750460.py:2: DtypeWarning: Columns (25,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\fryziggafl.csv')


afl_fantasy_score
player_team            player_first_name player_last_name player_name        age  year                   
Essendon               Zach              Merrett          Merrett, Zach      21.0 2016         117.590909
Sydney                 Errol             Gulden           Gulden, Errol      21.0 2023         112.250000
Gold Coast             Brayden           Fiorini          Fiorini, Brayden   19.0 2016         111.500000
Greater Western Sydney Tim               Taranto          Taranto, Tim       21.0 2019         111.307692
Carlton                Sam               Walsh            Walsh, Sam         21.0 2021         108.954545
Collingwood            Nick              Daicos           Daicos, Nick       20.0 2023         108.409091
Melbourne              Clayton           Oliver           Oliver, Clayton    21.0 2018         107.160000
Western Bulldogs       Jack              Macrae           Macrae, Jack       20.0 2014         104.380952
                                                                             21.0 2015         103.857143
Collingwood            Steele            Sidebottom       Sidebottom, Steele 21.0 2012         103.541667

# Data cleaning 


In [87]:
## Remove COVID impacted years. Apply TOG > 60%. Filter to only scores in decent weather conditions

joined_data= joined_data.query('year != 2019')\
        .query('year != 2020')\
        .query('time_on_ground_percentage > 60')\
        # .query('match_weather_type == ["MOSTLY_SUNNY", "OVERCAST","MOSTLY_CLEAR","SUNNY"]')

## Identify outlier scores (top / bottom 2 after applying first round of filters
joined_data['score_rank_asc'] = joined_data.groupby(['player_name', 'year'])['afl_fantasy_score'].rank(method="first", ascending=True)
joined_data['score_rank_desc'] = joined_data.groupby(['player_name', 'year'])['afl_fantasy_score'].rank(method="first", ascending=False)


matches_played = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_played=('year', 'mean'))

matches_played_min5 = matches_played.query('matches >=5')

joined_data = joined_data.merge(matches_played, on =['player_name','year'], how='inner')

# Remove outliers and seasons with fewer than 5 games

joined_data['Percentile'] = joined_data['score_rank_asc'] / joined_data['matches']

# joined_data = joined_data.query('0.15 <= Percentile <= 0.85')

joined_data

,Unnamed: 0.1,Unnamed: 0,venue_name,match_id,match_home_team,match_away_team,match_date,match_local_time,match_attendance,match_round,...,player_name,First Name,Last Name,dob,age,score_rank_asc,score_rank_desc,matches,year_played,Percentile
0,2939,600055,Gabba,14945,Brisbane Lions,Port Adelaide,30/07/2016,19:25:00,13085,19,...,"Smith, Archie",NaN,NaN,NaN,NaN,3.0,1.0,3,2016.0,1.000000
1,2940,600450,Adelaide Oval,14954,Adelaide,Brisbane Lions,6/08/2016,19:10:00,43549,20,...,"Smith, Archie",NaN,NaN,NaN,NaN,2.0,2.0,3,2016.0,0.666667
2,2941,600670,Gabba,14959,Brisbane Lions,Carlton,13/08/2016,13:45:00,17432,21,...,"Smith, Archie",NaN,NaN,NaN,NaN,1.0,3.0,3,2016.0,0.333333
3,2942,605993,MCG,15080,Collingwood,Brisbane Lions,28/05/2017,13:10:00,32750,10,...,"Smith, Archie",NaN,NaN,NaN,NaN,6.0,1.0,6,2017.0,1.000000
4,2943,606519,Gabba,15092,Brisbane Lions,Fremantle,10/06/2017,16:35:00,11742,12,...,"Smith, Archie",NaN,NaN,NaN,NaN,4.0,2.0,6,2017.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87877,667146,664771,Optus Stadium,16567,Fremantle,Port Adelaide,20/08/2023,14:40:00,0,23,...,"Emmett, Tom",Tom,Emmett,30/11/2001,22.0,1.0,2.0,2,2023.0,0.500000
87878,667147,664863,MCG,16569,Hawthorn,Fremantle,26/08/2023,13:45:00,0,24,...,"Emmett, Tom",Tom,Emmett,30/11/2001,22.0,2.0,1.0,2,2023.0,1.000000
87879,667149,665001,GMHBA Stadium,16572,Geelong,Western Bulldogs,26/08/2023,19:25:00,0,24,...,"Conway, Toby",Toby,Conway,24/04/2003,20.0,1.0,1.0,1,2023.0,1.000000
87880,667150,665092,Adelaide Oval,16574,Port Adelaide,Richmond,27/08/2023,12:00:00,0,24,...,"Trezise, James",James,Trezise,15/06/2002,21.0,1.0,1.0,1,2023.0,1.000000


# Seasons active factor
 

In [88]:
seasons_active = joined_data.groupby(['player_name']).agg(years_active=('year_played', 'nunique'))

seasons_active

,years_active
player_name,
"Aarts, Jake",2
"Abbott, Ryan",1
"Ablett, Gary",7
"Acres, Blake",8
"Adams, Leigh",3
...,...
"de Boer, Matt",9
"van Berlo, Nathan",4
"van Rooyen, Jacob",1


## In Season change factors

In [89]:
# ## Add scores during 1st half of season. Convert match round to numeric, return AFL score if valid, else return Null
joined_data['h1_fantasy_score'] = np.where(pd.to_numeric(joined_data['match_round'], errors='coerce') <= 10, joined_data['afl_fantasy_score'], np.NaN)
joined_data['h2_fantasy_score'] = np.where(pd.to_numeric(joined_data['match_round'], errors='coerce') > 10, joined_data['afl_fantasy_score'], np.NaN)


# Test
joined_data.agg({'afl_fantasy_score': 'mean','h1_fantasy_score': 'mean','h2_fantasy_score': 'mean'})

afl_fantasy_score    72.382456
h1_fantasy_score     72.448422
h2_fantasy_score     72.451418
dtype: float64

## In game stat factors

In [90]:
matches_played_min5.merge(joined_data.drop(['matches','year_played'], axis = 1), on =['player_name'], how='inner')

,player_name,matches,year_played,Unnamed: 0.1,Unnamed: 0,venue_name,match_id,match_home_team,match_away_team,match_date,...,year,First Name,Last Name,dob,age,score_rank_asc,score_rank_desc,Percentile,h1_fantasy_score,h2_fantasy_score
0,"Aarts, Jake",19,2021.0,661119,636665,MCG,15875,Richmond,Carlton,18/03/2021,...,2021,NaN,NaN,NaN,NaN,9.0,11.0,0.473684,48.0,NaN
1,"Aarts, Jake",19,2021.0,661120,637352,MCG,15890,Hawthorn,Richmond,28/03/2021,...,2021,NaN,NaN,NaN,NaN,1.0,19.0,0.052632,19.0,NaN
2,"Aarts, Jake",19,2021.0,661121,637626,MCG,15896,Richmond,Sydney,3/04/2021,...,2021,NaN,NaN,NaN,NaN,7.0,13.0,0.368421,44.0,NaN
3,"Aarts, Jake",19,2021.0,661122,637950,Adelaide Oval,15903,Port Adelaide,Richmond,9/04/2021,...,2021,NaN,NaN,NaN,NaN,2.0,18.0,0.105263,22.0,NaN
4,"Aarts, Jake",19,2021.0,661123,638318,Marvel Stadium,15911,St Kilda,Richmond,15/04/2021,...,2021,NaN,NaN,NaN,NaN,12.0,8.0,0.631579,55.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515859,"vandenBerg, Aaron",7,2018.0,631674,619927,MCG,15397,Melbourne,Greater Western Sydney,26/08/2018,...,2018,NaN,NaN,NaN,NaN,7.0,1.0,1.000000,NaN,103.0
515860,"vandenBerg, Aaron",7,2018.0,631675,620058,MCG,15400,Melbourne,Geelong,7/09/2018,...,2018,NaN,NaN,NaN,NaN,6.0,2.0,0.857143,NaN,NaN
515861,"vandenBerg, Aaron",7,2018.0,631676,620190,MCG,15403,Hawthorn,Melbourne,14/09/2018,...,2018,NaN,NaN,NaN,NaN,2.0,6.0,0.285714,NaN,NaN
515862,"vandenBerg, Aaron",7,2018.0,631677,620324,Optus Stadium,15406,West Coast,Melbourne,22/09/2018,...,2018,NaN,NaN,NaN,NaN,5.0,3.0,0.714286,NaN,NaN


In [91]:
#Explode player data by years so that each year matches to every other active year
player_summ_exp = matches_played_min5.merge(joined_data.drop(['matches','year_played'], axis = 1), on =['player_name'], how='inner')\
                

## Limit to games in the same year for 1yr factors, as well as 3 years up to the predicted season, and 5 years up to predicted season
player_summ_exp_1yr = player_summ_exp.query(('year_played == year'))
player_summ_exp_3yr = player_summ_exp.query(('year_played < year + 3')).query(('year_played >= year'))
player_summ_exp_5yr = player_summ_exp.query(('year_played < year + 5')).query(('year_played >= year'))

print(player_summ_exp.shape)
print(player_summ_exp_1yr.shape)
print(player_summ_exp_3yr.shape)
print(player_summ_exp_5yr.shape)

## Test matching works 
pd.set_option('display.max_rows', 50)
player_summ_exp_3yr.groupby(['year_played','year']).agg(countall=('player_name', 'count')).head(50)

(515864, 95)
(84996, 95)
(181868, 95)
(235629, 95)


countall
year_played year          
2012.0      2012      8191
2013.0      2012      7155
            2013      8192
2014.0      2012      6363
            2013      7002
            2014      8199
2015.0      2013      6327
            2014      7202
            2015      8215
2016.0      2014      6208
            2015      6885
            2016      8704
2017.0      2015      6430
            2016      7508
            2017      8731
2018.0      2016      6540
            2017      7440
            2018      8701
2021.0      2021      8661
2022.0      2021      7434
            2022      8573
2023.0      2021      6857
            2022      7521
            2023      8829

In [92]:
## Average factors over 1 year of game data
model_factors_agg_1yr = player_summ_exp_1yr.groupby(['player_name', 'year_played'])\
.agg({'player_height_cm': 'mean',
'player_weight_kg': 'mean',
'age': 'mean',
'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_1yr.columns = [str(col) + '_1yr' for col in model_factors_agg_1yr.columns]

model_factors_agg_1yr

player_height_cm_1yr  player_weight_kg_1yr  \
player_name       year_played                                               
Aarts, Jake       2021.0                      177.0                  76.0   
Ablett, Gary      2012.0                      182.0                  87.0   
                  2013.0                      182.0                  87.0   
                  2014.0                      182.0                  87.0   
                  2015.0                      182.0                  87.0   
...                                             ...                   ...   
van Berlo, Nathan 2015.0                        NaN                   NaN   
van Rooyen, Jacob 2023.0                      193.0                  96.0   
vandenBerg, Aaron 2015.0                      188.0                  94.0   
                  2016.0                      188.0                  94.0   
                  2018.0                      188.0                  94.0   

                               age_1yr  kicks_1yr  marks_1yr  handballs_1yr  \
player_name       year_played                                                 
Aarts, Jake       2021.0           NaN   6.263158   2.736842       5.210526   
Ablett, Gary      2012.0           NaN  19.450000   3.050000      14.300000   
                  2013.0           NaN  16.333333   3.476190      14.857143   
                  2014.0           NaN  17.071429   2.071429      15.928571   
                  2015.0           NaN  18.000000   2.000000       8.600000   
...                                ...        ...        ...            ...   
van Berlo, Nathan 2015.0           NaN   9.350000   3.950000       5.950000   
van Rooyen, Jacob 2023.0           NaN   4.578947   3.000000       4.052632   
vandenBerg, Aaron 2015.0           NaN  11.000000   3.416667       7.083333   
                  2016.0           NaN   5.714286   3.071429       8.928571   
                  2018.0           NaN   8.714286   4.285714       7.142857   

                               disposals_1yr  effective_disposals_1yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           11.473684                 8.684211   
Ablett, Gary      2012.0           33.750000                23.789474   
                  2013.0           31.190476                20.904762   
                  2014.0           33.000000                22.571429   
                  2015.0           26.600000                20.800000   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           15.300000                10.400000   
van Rooyen, Jacob 2023.0            8.631579                 6.684211   
vandenBerg, Aaron 2015.0           18.083333                10.666667   
                  2016.0           14.642857                 9.785714   
                  2018.0           15.857143                10.857143   

                               disposal_efficiency_percentage_1yr  goals_1yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                73.210526   0.947368   
Ablett, Gary      2012.0                                69.000000   1.300000   
                  2013.0                                67.238095   1.333333   
                  2014.0                                69.214286   1.714286   
                  2015.0                                78.600000   2.000000   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                68.350000   0.200000   
van Rooyen, Jacob 2023.0                                78.210526   1.421053   
vandenBerg, Aaron 2015.0                                57.500000   0.416667   
                  2016.0                                66.785714   0.857143   
                  2018.0                                69.571429   0.714286   

          

In [93]:
# Build 3 Year Player Metrics
model_factors_agg_3yr = player_summ_exp_3yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_3yr.columns = [str(col) + '_3yr' for col in model_factors_agg_3yr.columns]

model_factors_agg_3yr

kicks_3yr  marks_3yr  handballs_3yr  \
player_name       year_played                                        
Aarts, Jake       2021.0        6.263158   2.736842       5.210526   
Ablett, Gary      2012.0       19.450000   3.050000      14.300000   
                  2013.0       17.853659   3.268293      14.585366   
                  2014.0       17.654545   2.963636      14.927273   
                  2015.0       16.800000   2.800000      14.450000   
...                                  ...        ...            ...   
van Berlo, Nathan 2015.0       10.184211   3.815789       6.026316   
van Rooyen, Jacob 2023.0        4.578947   3.000000       4.052632   
vandenBerg, Aaron 2015.0       11.000000   3.416667       7.083333   
                  2016.0        8.153846   3.230769       8.076923   
                  2018.0        6.714286   3.476190       8.333333   

                               disposals_3yr  effective_disposals_3yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           11.473684                 8.684211   
Ablett, Gary      2012.0           33.750000                23.789474   
                  2013.0           32.439024                22.275000   
                  2014.0           32.581818                22.351852   
                  2015.0           31.250000                21.475000   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           16.210526                10.421053   
van Rooyen, Jacob 2023.0            8.631579                 6.684211   
vandenBerg, Aaron 2015.0           18.083333                10.666667   
                  2016.0           16.230769                10.192308   
                  2018.0           15.047619                10.142857   

                               disposal_efficiency_percentage_3yr  goals_3yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                73.210526   0.947368   
Ablett, Gary      2012.0                                69.000000   1.300000   
                  2013.0                                68.097561   1.317073   
                  2014.0                                68.381818   1.418182   
                  2015.0                                69.350000   1.550000   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                64.578947   0.315789   
van Rooyen, Jacob 2023.0                                78.210526   1.421053   
vandenBerg, Aaron 2015.0                                57.500000   0.416667   
                  2016.0                                62.500000   0.653846   
                  2018.0                                67.714286   0.809524   

                               behinds_3yr  hitouts_3yr  tackles_3yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2021.0          0.578947     0.000000     2.684211  ...   
Ablett, Gary      2012.0          0.950000     0.000000     5.500000  ...   
                  2013.0          1.024390     0.000000     5.024390  ...   
                  2014.0          0.890909     0.000000     5.327273  ...   
                  2015.0          0.825000     0.000000     5.250000  ...   
...                                    ...          ...          ...  ...   
van Berlo, Nathan 2015.0          0.394737     0.078947     3.578947  ...   
van Rooyen, Jacob 2023.0          0.421053     3.631579     2.210526  ...   
vandenBerg, Aaron 2015.0          0.916667     0.083333     5.333333  ...   
                  2016.0          0.769231     0.076923     4.384615  ...   
                  2018.0          0.714286     0.047619     3.428571  ...   

                               marks_on_lead_3yr  pressure_acts_3yr  \
player_name       year_played                           

In [94]:
# Build 5 Year Player Metrics

model_factors_agg_5yr = player_summ_exp_5yr.groupby(['player_name', 'year_played'])\
.agg({'kicks': 'mean',
'marks': 'mean',
'handballs': 'mean',
'handballs': 'mean',
'disposals': 'mean',
'effective_disposals': 'mean',
'disposal_efficiency_percentage': 'mean',
'goals': 'mean',
'behinds': 'mean',
'hitouts': 'mean',
'tackles': 'mean',
'rebounds': 'mean',
'inside_fifties': 'mean',
'clearances': 'mean',
'clangers': 'mean',
'free_kicks_for': 'mean',
'free_kicks_against': 'mean',
'brownlow_votes': 'mean',
'contested_possessions': 'mean',
'uncontested_possessions': 'mean',
'contested_marks': 'mean',
'marks_inside_fifty': 'mean',
'one_percenters': 'mean',
'bounces': 'mean',
'goal_assists': 'mean',
'time_on_ground_percentage': 'mean',
'centre_clearances': 'mean',
'stoppage_clearances': 'mean',
'score_involvements': 'mean',
'metres_gained': 'mean',
'turnovers': 'mean',
'intercepts': 'mean',
'tackles_inside_fifty': 'mean',
'contest_def_losses': 'mean',
'contest_def_one_on_ones': 'mean',
'contest_off_one_on_ones': 'mean',
'contest_off_wins': 'mean',
'def_half_pressure_acts': 'mean',
'effective_kicks': 'mean',
'f50_ground_ball_gets': 'mean',
'ground_ball_gets': 'mean',
'hitouts_to_advantage': 'mean',
'hitout_win_percentage': 'mean',
'intercept_marks': 'mean',
'marks_on_lead': 'mean',
'pressure_acts': 'mean',
'rating_points': 'mean',
'ruck_contests': 'mean',
'score_launches': 'mean',
'shots_at_goal': 'mean',
'spoils': 'mean',
'afl_fantasy_score': 'mean',
'h1_fantasy_score': 'mean',
'h2_fantasy_score': 'mean'
})

model_factors_agg_5yr.columns = [str(col) + '_5yr' for col in model_factors_agg_5yr.columns]

model_factors_agg_5yr

kicks_5yr  marks_5yr  handballs_5yr  \
player_name       year_played                                        
Aarts, Jake       2021.0        6.263158   2.736842       5.210526   
Ablett, Gary      2012.0       19.450000   3.050000      14.300000   
                  2013.0       17.853659   3.268293      14.585366   
                  2014.0       17.654545   2.963636      14.927273   
                  2015.0       17.683333   2.883333      14.400000   
...                                  ...        ...            ...   
van Berlo, Nathan 2015.0       11.000000   4.098361       6.327869   
van Rooyen, Jacob 2023.0        4.578947   3.000000       4.052632   
vandenBerg, Aaron 2015.0       11.000000   3.416667       7.083333   
                  2016.0        8.153846   3.230769       8.076923   
                  2018.0        8.272727   3.454545       7.878788   

                               disposals_5yr  effective_disposals_5yr  \
player_name       year_played                                           
Aarts, Jake       2021.0           11.473684                 8.684211   
Ablett, Gary      2012.0           33.750000                23.789474   
                  2013.0           32.439024                22.275000   
                  2014.0           32.581818                22.351852   
                  2015.0           32.083333                22.220339   
...                                      ...                      ...   
van Berlo, Nathan 2015.0           17.327869                11.283333   
van Rooyen, Jacob 2023.0            8.631579                 6.684211   
vandenBerg, Aaron 2015.0           18.083333                10.666667   
                  2016.0           16.230769                10.192308   
                  2018.0           16.151515                10.333333   

                               disposal_efficiency_percentage_5yr  goals_5yr  \
player_name       year_played                                                  
Aarts, Jake       2021.0                                73.210526   0.947368   
Ablett, Gary      2012.0                                69.000000   1.300000   
                  2013.0                                68.097561   1.317073   
                  2014.0                                68.381818   1.418182   
                  2015.0                                69.233333   1.466667   
...                                                           ...        ...   
van Berlo, Nathan 2015.0                                66.491803   0.409836   
van Rooyen, Jacob 2023.0                                78.210526   1.421053   
vandenBerg, Aaron 2015.0                                57.500000   0.416667   
                  2016.0                                62.500000   0.653846   
                  2018.0                                64.000000   0.666667   

                               behinds_5yr  hitouts_5yr  tackles_5yr  ...  \
player_name       year_played                                         ...   
Aarts, Jake       2021.0          0.578947     0.000000     2.684211  ...   
Ablett, Gary      2012.0          0.950000     0.000000     5.500000  ...   
                  2013.0          1.024390     0.000000     5.024390  ...   
                  2014.0          0.890909     0.000000     5.327273  ...   
                  2015.0          0.866667     0.000000     5.333333  ...   
...                                    ...          ...          ...  ...   
van Berlo, Nathan 2015.0          0.409836     0.065574     4.245902  ...   
van Rooyen, Jacob 2023.0          0.421053     3.631579     2.210526  ...   
vandenBerg, Aaron 2015.0          0.916667     0.083333     5.333333  ...   
                  2016.0          0.769231     0.076923     4.384615  ...   
                  2018.0          0.787879     0.060606     4.121212  ...   

                               marks_on_lead_5yr  pressure_acts_5yr  \
player_name       year_played                           

In [95]:
# Most common position for the year
# Calculate most common position played over past year
# Consider grouping if needed to remove specificity (e.g. Left and Right indicators probably not useful)


# Games player per year. Filter to only where games >= 5
# position_mapp = joined_data.groupby(['player_name','year']).agg({'match_date': 'count'})\
#     .query('match_date >=5')

position_data = joined_data

position_data['position_grouped'] = np.select(
    [
     position_data['player_position'].isin(["FB"]), 
     position_data['player_position'].isin(["BPL", "BPR"]),
     position_data['player_position'].isin(["CHB"]), 
     position_data['player_position'].isin(["HBFL", "HBFR"]),
     position_data['player_position'].isin(["RK"]), 
     position_data['player_position'].isin(["C", "RR" ,"R"]),
     position_data['player_position'].isin(["WL", "WR"]), 
     position_data['player_position'].isin(["FF"]),
     position_data['player_position'].isin(["FPL", "FPR"]),
     position_data['player_position'].isin(["HFFL", "HFFR"]),
     position_data['player_position'].isin(["CHF"]),
     position_data['player_position'].isin(["INT", "SUB"])
    ],
    [
      'FB',
      'BP',
      'CHB',
      'HBF',
      'RUCK',
      'C',
      'W',
      'FF',
      'FP',
      'HFF',
      'CHF',
      'Ignore'  
    ], 
    default='Other'
     )

## Test no positions have not been allocated. Result = 0 rows
# joined_data.query('position_grouped in ["Other"]')\
#     .groupby(['player_position']).agg(matches=('match_date', 'count'))

position_data['matches'] = position_data.groupby(['player_name', 'year','position_grouped'])['match_id'].transform('count')

# Find most common position, ignoring sub and interchange scores
position_data['rank'] = position_data.query('position_grouped not in ["Ignore"]').groupby(['player_name', 'year'])['matches'].rank(method="first", ascending=False)

player_pos_yearly = position_data.query('rank == 1')[['player_name', 'year','position_grouped']]

player_pos_yearly['position_grouped_simple'] = np.select(
    [
     player_pos_yearly['position_grouped'].isin(["FB", "BP","FF","FP"]), 
     player_pos_yearly['position_grouped'].isin(["CHB","CHF","C"]), 
     player_pos_yearly['position_grouped'].isin(["HBF", "W", "HFF"]),
     player_pos_yearly['position_grouped'].isin(["RUCK"])
    ],
    [
      'Key_Position',
      'Centres',
      'Wings_Flank',
      'Ruck'
    ], 
    default='Other'
     )

# Test output
player_pos_yearly.query('player_name == "Himmelberg, Harry"')

,player_name,year,position_grouped,position_grouped_simple
71170,"Himmelberg, Harry",2016,FP,Key_Position
71175,"Himmelberg, Harry",2017,FP,Key_Position
71185,"Himmelberg, Harry",2018,FF,Key_Position
71221,"Himmelberg, Harry",2021,CHF,Centres
71233,"Himmelberg, Harry",2022,CHF,Centres
71266,"Himmelberg, Harry",2023,CHB,Centres


## Response variable

In [96]:
# Average AFL fantasy score per season
response_var = joined_data.groupby(['player_name','year']).agg(matches=('match_date', 'count'),year_int=('year', 'mean'),avg_score=('afl_fantasy_score', 'mean'))\
    .query('matches >= 5').query('avg_score > 0')\
    .sort_values(by=['avg_score'], ascending=False)

# Subtract 1 year from AFL score so it matches to factors from last year
response_var['predicted_year'] = response_var['year_int'] - 1

response_var  

matches  year_int   avg_score  predicted_year
player_name     year                                               
Rockliff, Tom   2014       18    2014.0  134.777778          2013.0
Swan, Dane      2012       21    2012.0  130.666667          2011.0
Rockliff, Tom   2016       15    2016.0  128.400000          2015.0
Mitchell, Tom   2018       24    2018.0  128.166667          2017.0
                2017       22    2017.0  127.181818          2016.0
...                       ...       ...         ...             ...
Gardner, Ryan   2021        7    2021.0   29.000000          2020.0
Spicer, Phoenix 2022        5    2022.0   27.000000          2021.0
Blanck, James   2023       13    2023.0   25.307692          2022.0
Cadman, Aaron   2023        7    2023.0   24.142857          2022.0
Brown, Nathan   2017       22    2017.0   23.136364          2016.0

[5289 rows x 4 columns]

## Join all factors together


In [97]:
## Player / year combos to include
factors = player_summ.merge(seasons_active, left_on=['player_name'], right_on=['player_name'], how='inner')\
           .merge(player_pos_yearly, left_on=['player_name', 'year_played'], right_on=['player_name', 'year'], how='inner')\
           .merge(model_factors_agg_1yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_3yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(model_factors_agg_5yr, left_on=['player_name', 'year_played'], right_on=['player_name', 'year_played'], how='inner')\
           .merge(response_var, left_on=['player_name', 'year_played'], right_on=['player_name', 'predicted_year'], how='left')\


## Add delta factors. Use lazy difference between 1 year and 3 year average

factors['fantasy_score_yearly_change'] = (factors['afl_fantasy_score_1yr'] - factors['afl_fantasy_score_3yr']).fillna(0)
factors['fantasy_score_inyear_change'] = (factors['h2_fantasy_score_1yr'] - factors['h1_fantasy_score_1yr']).fillna(0)




## Create factor that scales up young players based on expected YoY growth

In [100]:
## Calculate average season growth based on player age

player_growth = factors.dropna(axis= 0, how='any').groupby(['age_1yr']).agg({'fantasy_score_yearly_change': 'mean'})
player_growth_pos = factors.dropna(axis= 0, how='any').groupby(['age_1yr', 'position_grouped_simple']).agg({'fantasy_score_yearly_change': 'mean'})

player_growth_pos.rename(columns={'fantasy_score_yearly_change':'fantasy_score_yearly_change_exp'}, inplace=True)

# Add to factors
factors = factors.merge(player_growth_pos, on=['age_1yr', 'position_grouped_simple'], how='left')\

# Fill nulls with 0
factors['fantasy_score_yearly_change_exp'] = (factors['fantasy_score_yearly_change_exp']).fillna(0)

# Fill blank ages with 26, AFL average
factors['age_1yr'] = (factors['age_1yr']).fillna(26)

player_growth_pos

KeyError: 'fantasy_score_yearly_change_exp'

In [104]:
player_growth.head(100)

,fantasy_score_yearly_change
age_1yr,
19.0,0.000000
20.0,1.413972
21.0,2.815553
22.0,3.446957
23.0,1.491049
24.0,1.397633
25.0,0.926172
26.0,0.923742
27.0,2.218640


# Export factors

In [99]:
print(factors.shape)
print(factors.head(10))
factors.to_csv(r'C:\Users\anoble\Git\Outside_Work\AFL Fantasy\Data\Inputs\factors.csv',mode='w+')

(3226, 176)
    player_name  matches_x  year_played  years_active  year position_grouped  \
0   Aarts, Jake         12       2021.0             2  2021              HFF   
1  Ablett, Gary         18       2012.0             7  2012                C   
2  Ablett, Gary         18       2013.0             7  2013                C   
3  Ablett, Gary         12       2014.0             7  2014                C   
4  Ablett, Gary         10       2017.0             7  2017                C   
5  Ablett, Gary         12       2018.0             7  2018                C   
6  Acres, Blake         12       2017.0             8  2017               FP   
7  Acres, Blake         14       2021.0             8  2021                W   
8  Acres, Blake         17       2022.0             8  2022                W   
9  Acres, Blake         13       2023.0             8  2023                W   

  position_grouped_simple  player_height_cm_1yr  player_weight_kg_1yr  \
0             Wings_Flank         